In [1]:
import numpy as np, pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import collections
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gujarati-train.csv')

In [20]:
def count_length():
    data['word_count'] = data['headline'].apply(lambda x: len(str(x).split(" ")))

In [21]:
count_length()

In [22]:
data.head(10)

,headline,Label,word_count
12340,Googleની આ એપ્લિકેશનને પ્રમોટ કરવા પર મળશે કેશ...,Tech,11
8940,"હવે, જૂના ફોટો પણ હસતા જોવા મળશે, આવી ગઈ છે નવ...",Tech,12
6002,"એબી ડી વિલિયર્સ: તે સ્પાઇડરમેન હતો, કોઈ બેટ્સમ...",Sports,14
24715,અમરિન્દરસિંહ ઇસીને મેજિથિયાના અભિયાનમાં પ્રતિબ...,Politics,8
30032,"મુવાહ, દિલ્હીમાં શમુ નાગા શાંતિ વાટાઘાટો ફરી શ...",Politics,10
22151,કોરોના પ્રભાવ : ઇન્ફોસિસે પ્રમોશન અને વેતનવધાર...,Business,10
10027,"PUBG પર પ્રતિબંધ લાગ્યો તો શું થયું, આ Made In...",Tech,18
37167,ચાલું ઈન્ટરવ્યૂમાં પ્રેમ ઉભરાતાં રણવીરે દીપિકા...,Entertainment,13
1960,પુણેને 1 રનથી હરાવી મુંબઇ બની IPL ચેમ્પિયન,Sports,8
13590,"ભૂલી જશો ડ્યૂઅલ કેમેરા, આવી રહ્યો છે 9 કેમેરાવ...",Tech,10


In [23]:
import re
def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text

In [24]:
for i in range(len(data)):
    data['headline'][i] = processText(data['headline'][i])

In [25]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
data['headline'] = data['headline'].apply(lambda x: tokenization(x))

In [26]:
for i in range(len(data)):
    data['headline'][i] = [s.replace("\n", "") for s in data['headline'][i]]

In [27]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))

In [28]:
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
to_be_removed = stopwords_gu + punctuations + stopwords_en

In [29]:
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (to_be_removed)]
count_length()
data.head(10)

,headline,Label,word_count
12340,"[એપ્લિકેશનને, પ્રમોટ, કરવા, મળશે, કેશેબેક, કંપ...",Tech,8
8940,"[જૂના, ફોટો, હસતા, જોવા, મળશે, આવી, ગઈ, એપ]",Tech,8
6002,"[એબી, વિલિયર્સ, સ્પાઇડરમેન, હતો, બેટ્સમેને, તે...",Sports,9
24715,"[અમરિન્દરસિંહ, ઇસીને, મેજિથિયાના, અભિયાનમાં, પ...",Politics,7
30032,"[મુવાહ, દિલ્હીમાં, શમુ, નાગા, શાંતિ, વાટાઘાટો,...",Politics,8
22151,"[કોરોના, પ્રભાવ, ઇન્ફોસિસે, પ્રમોશન, વેતનવધારા...",Business,8
10027,"[પ્રતિબંધ, લાગ્યો, તો, થયું, ગેમ્સ, ટ્રાય, કરશ...",Tech,10
37167,"[ચાલું, ઈન્ટરવ્યૂમાં, પ્રેમ, ઉભરાતાં, રણવીરે, ...",Entertainment,10
1960,"[પુણેને, રનથી, હરાવી, મુંબઇ, ચેમ્પિયન]",Sports,5
13590,"[ભૂલી, જશો, ડ્યૂઅલ, કેમેરા, આવી, રહ્યો, કેમેરા...",Tech,8


In [30]:
corpus_list =[]
for i in range(len(data)):
    corpus_list +=data['headline'][i]
counter=collections.Counter(corpus_list)

In [31]:
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (least_common)]

In [32]:
data['headline'] = data['headline'].str.join(" ")
data.head(10)

,headline,Label,word_count
12340,એપ્લિકેશનને પ્રમોટ કરવા મળશે કેશેબેક કંપનીએ બન...,Tech,8
8940,જૂના ફોટો હસતા જોવા મળશે આવી ગઈ એપ,Tech,8
6002,એબી વિલિયર્સ સ્પાઇડરમેન હતો બેટ્સમેને તેની પાસ...,Sports,9
24715,અમરિન્દરસિંહ ઇસીને મેજિથિયાના અભિયાનમાં પ્રતિબ...,Politics,7
30032,મુવાહ દિલ્હીમાં શમુ નાગા શાંતિ વાટાઘાટો શરૂ કરવા,Politics,8
22151,કોરોના પ્રભાવ ઇન્ફોસિસે પ્રમોશન વેતનવધારા હાલ ...,Business,8
10027,પ્રતિબંધ લાગ્યો તો થયું ગેમ્સ ટ્રાય કરશો તો થા...,Tech,10
37167,ચાલું ઈન્ટરવ્યૂમાં પ્રેમ ઉભરાતાં રણવીરે દીપિકા...,Entertainment,10
1960,પુણેને રનથી હરાવી મુંબઇ ચેમ્પિયન,Sports,5
13590,ભૂલી જશો ડ્યૂઅલ કેમેરા આવી રહ્યો કેમેરાવાળો સ્...,Tech,8


In [16]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['headline'],data['Label'],test_size=0.2,random_state=42)

In [17]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Train_Y[0:10])
print(Train_X[0:10])

[3 4 2 1 1 4 0 0 3 0]
7178     જમ્મુ ઘડિયાળથી વર્ષીય આઇપીએલની શરૂઆતથી કિલોમીટ...
9370                          મજાક ઉડાવનાર જ ફોનમાં ચાર્જર
27303    અરુણાચલ વિરોધ રાજ્યસભામાં વિક્ષેપ કરે કોંગ લોક...
35490    ‘સુશાંતના મોત કોનો હાથ જાણું છું’ કહેનાર શેખર ...
34340    ક્ષિતિજ પ્રસાદની જામીન અરજી થશે સુનાવણી ડ્રગ્સ...
8087         ૧૨૦ની સ્પીડે ઊડતી ભારતીય હાઈબ્રિડ ફ્લાઇંગ કાર
20759    લાખ કામદારો સારા સમાચાર બેકારીના લાભરૂપે સરકાર...
18804    અમેઝોન મળતા ગાયનાં છાણાને વ્યક્તિ કેક સમજીને ખ...
4201     પ્રીમિયર લીગ બ્રેન્ટફોર્ડ મેન યુનાઈટેડ રાઇડ નસ...
16250    ખાતાધારક ધ્યાન આગામી મહિનાથી બેંકમાં લાગૂ થઇ જ...
Name: headline, dtype: object


In [18]:
Tfidf_vect = TfidfVectorizer() #5000
Tfidf_vect.fit(data['headline'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [19]:
dt = RandomForestClassifier(max_depth = 250 ,n_estimators = 1000) #2500
dt.fit(Train_X_Tfidf,Train_Y)
predictions_dec = dt.predict(Test_X_Tfidf)

In [20]:
print("RFC Accuracy Score -> ",accuracy_score(predictions_dec, Test_Y)*100)

RFC Accuracy Score ->  72.67072525145579


In [21]:
mat = confusion_matrix(predictions_dec, Test_Y)
mat

array([[ 988,   67,   78,   68,  161],
       [  90, 1095,   67,  119,   80],
       [  96,  121, 1207,  185,   72],
       [ 118,  116,  127, 1132,   83],
       [ 199,   78,   64,   76, 1069]], dtype=int64)

In [22]:
report = classification_report(predictions_dec, Test_Y)
print(report)

              precision    recall  f1-score   support

           0       0.66      0.73      0.69      1362
           1       0.74      0.75      0.75      1451
           2       0.78      0.72      0.75      1681
           3       0.72      0.72      0.72      1576
           4       0.73      0.72      0.72      1486

    accuracy                           0.73      7556
   macro avg       0.73      0.73      0.73      7556
weighted avg       0.73      0.73      0.73      7556



In [23]:
import joblib
import os
loaded_model = joblib.dump(dt, 'GUJ_rfc_final.pkl')

In [24]:
simple_test = ["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred1 = dt.predict(Train_X_Tfidf)
pred1[0]
predictions_test = Encoder.inverse_transform(pred1)
predictions_test[0]

'Business'

In [25]:
simple_test2 = ["SRKના દીકરાની ઘરવાપસી LIVE:આર્યન ખાનને 'મન્નત'થી આર્થર રોડ જેલનું 13 કિલોમીટરનું અંતર કાપતાં 28 દિવસ થયા"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test2)
pred2 = dt.predict(Train_X_Tfidf)
pred2[0]
predictions_test = Encoder.inverse_transform(pred2)
predictions_test[0]

'Entertainment'